# Project 1 : Prepare environment for Sparkify

<img src="https://wiki.postgresql.org/images/a/a4/PostgreSQL_logo.3colors.svg" width="250" height="250">

<center><h1><span style='color:blue'>Environment preparation</span></h1></center>

Udacity environment has been prepared to ease student task, i.e. has a Postgres instance available for training exercises.

Let's create one based on Kubernetes.

* Add Pyscopg2 module to Python
* Load in K8s Postgresql

In [ ]:
# Load package
#!pip install psycopg2-binary
#!pip install pandas --upgrade
#!pip install sqlalchemy --upgrade # ORM for databases
#!pip install ipython-sql --upgrade # SQL magic function

<h3><span style='color:blue'>Using K8S PostgreSQL</span></h3>

Obviously you need a k8s avaible like: Minikube, Minishift, Docker (with K8s)

Helm is need to, go to [helm.sh](http://helm.sh)

In [1]:
from time import sleep
import os

In [2]:
helm_version = !helm version --short
assert helm_version[0][:2] == 'v3', "Expected HELM version not available, visit https://helm.sh"

#!curl -fsSL -o /tmp/get_helm.sh https://raw.githubusercontent.com/helm/helm/master/scripts/get-helm-3
#!chmod 700 /tmp/get_helm.sh
#!ls -al /tmp/
#!./get_helm.sh

In [3]:
!helm repo add bitnami https://charts.bitnami.com/bitnami

"bitnami" has been added to your repositories


In [4]:
CHART_INSTANCE_NAME = 'dend-p1-sparkify'
os.environ['postgresql_port_instance_name'] = CHART_INSTANCE_NAME + "-postgresql"
os.getenv('postgresql_port_instance_name')

'dend-p1-sparkify-postgresql'

Let's write a YAML config file for Helm

Reference: https://github.com/helm/charts/tree/master/stable/postgresql

In [5]:
%%writefile config_helm_pgsql.yaml
service:
    type: NodePort
    nodePort: 30432
postgresqlUsername: postgres
postgresqlPassword: password

Overwriting config_helm_pgsql.yaml


In [6]:
helm_chart_out = !helm install {CHART_INSTANCE_NAME} stable/postgresql -f config_helm_pgsql.yaml

In [7]:
postgresql_port_forward_command = helm_chart_out[-2].strip()
os.environ['postgresql_port_forward_command'] = postgresql_port_forward_command
os.getenv('postgresql_port_forward_command')

'export NODE_PORT=$(kubectl get --namespace default -o jsonpath="{.spec.ports[0].nodePort}" services dend-p1-sparkify-postgresql)'

In [8]:
# Waits until postgresl is running on 
max_checks_postgresql_run = 20

!kubectl get pods

while max_checks_postgresql_run > 0:

    postgres_is_running = !kubectl get pods|fgrep {CHART_INSTANCE_NAME}|fgrep "1/1"|fgrep "Running"
    
    if len(postgres_is_running) > 0 and not postgres_is_running[0] == 'No resources found.':
        break
    else:
        sleep(5)

        max_checks_postgresql_run -= 1

!kubectl get pods
assert max_checks_postgresql_run > 0, "Probably Postgresql is not running"

NAME                            READY   STATUS    RESTARTS   AGE
dend-p1-sparkify-postgresql-0   0/1     Pending   0          1s
NAME                            READY   STATUS    RESTARTS   AGE
dend-p1-sparkify-postgresql-0   1/1     Running   0          27s


In [9]:
!kubectl get po,svc,pv

NAME                                READY   STATUS    RESTARTS   AGE
pod/dend-p1-sparkify-postgresql-0   1/1     Running   0          27s

NAME                                           TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)          AGE
service/dend-p1-sparkify-postgresql            NodePort    10.111.69.5   <none>        5432:30432/TCP   27s
service/dend-p1-sparkify-postgresql-headless   ClusterIP   None          <none>        5432/TCP         27s
service/kubernetes                             ClusterIP   10.96.0.1     <none>        443/TCP          21d

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                        STORAGECLASS   REASON   AGE
persistentvolume/pvc-520df70d-1125-4b3f-a84b-356275308814   8Gi        RWO            Delete           Bound    default/data-dend-p1-sparkify-postgresql-0   hostpath                26s


In [10]:
# Getting console command to connect with current instance of postgres
k8s_psql_command = helm_chart_out[19].strip().replace("$POSTGRES_PASSWORD", "password") + " -c "
print(k8s_psql_command)

kubectl run dend-p1-sparkify-postgresql-client --rm --tty -i --restart='Never' --namespace default --image docker.io/bitnami/postgresql:11.6.0-debian-9-r0 --env="PGPASSWORD=password" --command -- psql --host dend-p1-sparkify-postgresql -U postgres -d postgres -p 5432 -c 


<h3><span style='color:blue'>Check Postgresql availibity</span></h3>
We had created an Postgresql on a K8s infraestructure, next we will test if it is avaiable

In [11]:
# Checks postgresql connection
select_1_postgresql_out = !{k8s_psql_command} 'SELECT 1;'
assert len(select_1_postgresql_out) > 0, 'Postgresql -select 1- failed, check it'
!{k8s_psql_command} 'SELECT version();'

                                                 version                        
                         
--------------------------------------------------------------------------------
-------------------------
 PostgreSQL 11.6 on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18+deb9u1
) 6.3.0 20170516, 64-bit
(1 row)

pod "dend-p1-sparkify-postgresql-client" deleted


<h3><span style='color:blue'>Initialize Postgresql Sparkify DB for project #1</span></h3>

In [12]:
!{k8s_psql_command} "CREATE ROLE sparkify WITH LOGIN ENCRYPTED PASSWORD 'sparkify'"

CREATE ROLE
pod "dend-p1-sparkify-postgresql-client" deleted


In [13]:
!{k8s_psql_command} 'alter user sparkify createdb;'

ALTER ROLE
pod "dend-p1-sparkify-postgresql-client" deleted


In [14]:
!{k8s_psql_command} 'create database sparkifydb WITH ENCODING utf8;'

If you don't see a command prompt, try pressing enter.
CREATE DATABASE
pod "dend-p1-sparkify-postgresql-client" deleted


In [15]:
!{k8s_psql_command} 'grant all privileges on database sparkifydb to sparkify;'

GRANT
pod "dend-p1-sparkify-postgresql-client" deleted


In [16]:
!{k8s_psql_command} 'SELECT usename, usecreatedb FROM pg_user;'

 usename  | usecreatedb 
----------+-------------
 postgres | t
 sparkify | t
(2 rows)

pod "dend-p1-sparkify-postgresql-client" deleted
